In [ ]:
import torch
from torch_geometric.data import Batch
import numpy as np
from torch_geometric.utils import to_networkx
import networkx as nx

In [ ]:
batch_graph = torch.load('plots/graph_100.pt')

In [ ]:
original_graph = Batch.to_data_list(batch_graph.org)[1]

In [ ]:
rewired = batch_graph.candidates[0]

In [ ]:
rewired = Batch.to_data_list(rewired)[1]

In [ ]:
mask = rewired.edge_weight.nonzero().flatten()

In [ ]:
rewired.edge_index = rewired.edge_index[:, mask]

In [ ]:
layout = {i: pos for i, pos in enumerate(original_graph.nx_layout.cpu().numpy())}

In [ ]:
rewired_nx = to_networkx(rewired, remove_self_loops=True)
org_nx = to_networkx(original_graph, remove_self_loops=True)

In [ ]:
# https://matplotlib.org/stable/tutorials/colors/colors.html

In [ ]:
node_colors_int = rewired.x[:, 1].detach().cpu().squeeze()
leaf_1 = torch.where(node_colors_int == 1)[0].numpy()
root = np.zeros(1, dtype=np.int64)
intermediate = np.array([1,2,3,6,9,10,13,17,18,21,16,24,25,28])
leaf_0 = np.setdiff1d(np.arange(31), np.union1d(np.union1d(root, intermediate), leaf_1))

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(8, 4))
axs[0].axis('off')
axs[0].axis('equal')
nx.draw_networkx_nodes(org_nx, pos=layout, nodelist=root, node_color='black', ax=axs[0])
nx.draw_networkx_edges(org_nx, pos=layout, edgelist=list(org_nx.edges), width=1, edge_color='k', ax=axs[0], arrowsize=7)
nx.draw_networkx_nodes(org_nx, pos=layout, nodelist=leaf_1, node_color='#FFD700', ax=axs[0], node_shape='h')
nx.draw_networkx_nodes(org_nx, pos=layout, nodelist=leaf_0, node_color='#7FFFD4', ax=axs[0], node_shape='s')
nx.draw_networkx_nodes(org_nx, pos=layout, nodelist=intermediate, node_color='grey', ax=axs[0], node_shape='o')

axs[1].axis('off')
axs[1].axis('equal')
nx.draw_networkx_nodes(rewired_nx, pos=layout, nodelist=root, node_color='black', ax=axs[1])
nx.draw_networkx_edges(rewired_nx, pos=layout, edgelist=list(rewired_nx.edges), width=1, edge_color='k', ax=axs[1], arrowsize=7)
nx.draw_networkx_nodes(rewired_nx, pos=layout, nodelist=leaf_1, node_color='#FFD700', ax=axs[1], node_shape='h')
nx.draw_networkx_nodes(rewired_nx, pos=layout, nodelist=leaf_0, node_color='#7FFFD4', ax=axs[1], node_shape='s')
nx.draw_networkx_nodes(rewired_nx, pos=layout, nodelist=intermediate, node_color='grey', ax=axs[1], node_shape='o')
fig.tight_layout()

plt.savefig('leafcount.pdf', dpi=500)